In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
import gensim
import spacy
import pyspark
from pyspark import *

//anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [79]:
tweets_data = []
#tweets_file = open(tweets_data_path, "r")

with open('/Users/uun466/Desktop/Data-Science-Project/tweet_file.json') as json_data:
    d = json.load(json_data)
    if len(d) > 1:
        for each in range(len(d)):
            tweets_data.append(d[each])
    else:
        tweets_data.append(d)

In [81]:
print len(tweets_data)

20


In [80]:
tweets_data

[{u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Mon Nov 07 01:40:22 +0000 2016',
  u'entities': {u'hashtags': [{u'indices': [19, 37],
     u'text': u'TheGirlOnTheTrain'},
    {u'indices': [38, 51], u'text': u'screenwriter'}],
   u'symbols': [],
   u'urls': [],
   u'user_mentions': [{u'id': 14807513,
     u'id_str': u'14807513',
     u'indices': [3, 17],
     u'name': u'Final Draft',
     u'screen_name': u'finaldraftinc'}]},
  u'favorite_count': 0,
  u'favorited': False,
  u'geo': None,
  u'id': 795440728558145536,
  u'id_str': u'795440728558145536',
  u'in_reply_to_screen_name': None,
  u'in_reply_to_status_id': None,
  u'in_reply_to_status_id_str': None,
  u'in_reply_to_user_id': None,
  u'in_reply_to_user_id_str': None,
  u'is_quote_status': False,
  u'lang': u'en',
  u'metadata': {u'iso_language_code': u'en', u'result_type': u'recent'},
  u'place': None,
  u'retweet_count': 3,
  u'retweeted': False,
  u'retweeted_status': {u'contributors': None,
   u'coordinates'

In [84]:
tweets = pd.DataFrame()

In [86]:
tweets['text'] = map(lambda tweet: tweet['text'].strip(), tweets_data)
tweets['lang'] = map(lambda tweet: tweet['lang'], tweets_data)
tweets['country'] = map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data)
tweets['user_nm'] = map(lambda tweet: tweet['user']['name'].encode('utf-8'), tweets_data)
tweets['coordinates'] = map(lambda tweet: tweet['coordinates'], tweets_data)
tweets['location'] = map(lambda tweet: tweet['user']['location'], tweets_data)
tweets['retweets_count'] = map(lambda tweet: tweet['retweet_count'], tweets_data)
#tweets['retweet_location'] = map(lambda tweet: tweet['retweeted_status']['user']['location'], tweets_data)

In [87]:
tweets['retweets_count']

0      3
1      0
2     35
3     62
4      0
5      0
6      1
7      0
8      0
9      0
10     0
11     3
12     3
13     3
14     0
15     0
16     0
17     2
18     1
19     0
Name: retweets_count, dtype: int64

In [ ]:
%matplotlib inline

In [88]:
tweets

,text,lang,country,user_nm,coordinates,location,retweets_count
0,RT @finaldraftinc: #TheGirlOnTheTrain #screenw...,en,None,Ian Jon McCoy,None,"Atlanta, Georgia",3
1,"I don't understand the movie ""The Girl on the ...",en,None,Bourlart Noemie,None,"Lille, France",0
2,RT @BritishVogue: Have you seen #TheGirlOnTheT...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",35
3,RT @girlontrainfilm: #TheGirlOnTheTrain is num...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",62
4,#thegirlonthetrain #rainydayread #thankyouomeg...,en,None,Alle Davis,None,"New Plymouth District, New Zea",0
5,Just saw #TheGirlOnTheTrain Emily Blunt's perf...,en,None,Heather K.,None,"North Carolina, USA",0
6,RT @MissCharleyF: Wow! @PaulaHWrites' #TheGirl...,en,None,craig a ball,None,,1
7,Finally saw #TheGirlOnTheTrain @TGOTTfilm @emi...,en,None,Carrie Tyrie,None,"Scotland, United Kingdom",0
8,yay almost done w reading my book! took advant...,en,None,Saritah,None,AZ,0
9,"Loved, loved the book: #thegirlonthetrain Jus...",en,None,Sheila Tidwell,None,"Nashville, TN",0


In [89]:
## Get Verbatim text
verbatim = [str(v.encode('utf-8')) for v in tweets.text.values.tolist()]

In [90]:
verbatim

['RT @finaldraftinc: #TheGirlOnTheTrain #screenwriter Erin Cressida Wilson takes us behind the scenes of the hit thriller! Listen here: https\xe2\x80\xa6',
 'I don\'t understand the movie "The Girl on the Train" takes place in New-York \n#TheGirlOnTheTrain',
 'RT @BritishVogue: Have you seen #TheGirlOnTheTrain yet? Here\xe2\x80\x99s what you need to know before watching the film: https://t.co/4S90NcJZqL http\xe2\x80\xa6',
 'RT @girlontrainfilm: #TheGirlOnTheTrain is number 1 film in the UK. Have you seen it yet? https://t.co/pXE8x0E8ZK',
 "#thegirlonthetrain #rainydayread #thankyouomegledr bekah's day is about to b made. #decoyname #AlleHartley\xe2\x80\xa6 https://t.co/1eCMGc8fZx",
 "Just saw #TheGirlOnTheTrain Emily Blunt's performance was amazing!",
 "RT @MissCharleyF: Wow! @PaulaHWrites' #TheGirlOnTheTrain is an outstanding thriller! Full of twists and turns with suspense. A complete pag\xe2\x80\xa6",
 "Finally saw #TheGirlOnTheTrain @TGOTTfilm @emilyblunt was amazing, it's just how

In [91]:
import textauger
import re

In [92]:
from textauger import preprocessing
from nltk.sentiment.vader import SentimentIntensityAnalyzer as Vader
from textauger import textfeatures

//anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [93]:
tweets['text_clean'] = [re.sub(r"http\S+", "", v) for v in tweets.text.values.tolist()]
tweets['text_clean'] = [re.sub(r"#\S+", "", v) for v in tweets.text_clean.values.tolist()]
tweets['text_clean'] = [re.sub(r"@\S+", "", v) for v in tweets.text_clean.values.tolist()]
tweets['text_clean'] = [re.sub(r"u'RT\S+", "", v) for v in tweets.text_clean.values.tolist()]
tweets['text'] = [v.replace('\n'," ") for v in tweets.text.values.tolist()]

In [94]:
tweets['text_clean'] = preprocessing.clean_text(text=tweets.text_clean.values, 
                         remove_short_tokens_flag=False,  
                         lemmatize_flag=True)

In [ ]:
tweets['text_clean'][1]

In [95]:
tweets['sentiment_score'] = [textfeatures.score_sentiment(v)['compound'] for v in tweets.text_clean.values.tolist()]

In [96]:
tweets

,text,lang,country,user_nm,coordinates,location,retweets_count,text_clean,sentiment_score
0,RT @finaldraftinc: #TheGirlOnTheTrain #screenw...,en,None,Ian Jon McCoy,None,"Atlanta, Georgia",3,rt erin cressida wilson take u behind scene hi...,0.1027
1,"I don't understand the movie ""The Girl on the ...",en,None,Bourlart Noemie,None,"Lille, France",0,dont understand movie girl train take place ne...,0.0000
2,RT @BritishVogue: Have you seen #TheGirlOnTheT...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",35,rt seen yet here s need know watching film,0.0000
3,RT @girlontrainfilm: #TheGirlOnTheTrain is num...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",62,rt number film uk seen yet,0.0772
4,#thegirlonthetrain #rainydayread #thankyouomeg...,en,None,Alle Davis,None,"New Plymouth District, New Zea",0,bekahs day b made,0.0000
5,Just saw #TheGirlOnTheTrain Emily Blunt's perf...,en,None,Heather K.,None,"North Carolina, USA",0,saw emily blunts performance amazing,0.5859
6,RT @MissCharleyF: Wow! @PaulaHWrites' #TheGirl...,en,None,craig a ball,None,,1,rt wow outstanding thriller full twist turn su...,0.8481
7,Finally saw #TheGirlOnTheTrain @TGOTTfilm @emi...,en,None,Carrie Tyrie,None,"Scotland, United Kingdom",0,finally saw amazing imagined,0.5859
8,yay almost done w reading my book! took advant...,en,None,Saritah,None,AZ,0,yay almost done w reading book took advantage ...,0.6597
9,"Loved, loved the book: #thegirlonthetrain Jus...",en,None,Sheila Tidwell,None,"Nashville, TN",0,loved loved book saw movie and,0.8316


In [ ]:
textfeatures.score_sentiment(tweets['text_clean'][1])

In [97]:
tweets.loc[tweets['sentiment_score'] > 0.0, 'sentiment'] = 'positive'
tweets.loc[tweets['sentiment_score'] == 0.0, 'sentiment'] = 'neutral'
tweets.loc[tweets['sentiment_score'] < 0.0, 'sentiment'] = 'negative'

In [98]:
tweets

,text,lang,country,user_nm,coordinates,location,retweets_count,text_clean,sentiment_score,sentiment
0,RT @finaldraftinc: #TheGirlOnTheTrain #screenw...,en,None,Ian Jon McCoy,None,"Atlanta, Georgia",3,rt erin cressida wilson take u behind scene hi...,0.1027,positive
1,"I don't understand the movie ""The Girl on the ...",en,None,Bourlart Noemie,None,"Lille, France",0,dont understand movie girl train take place ne...,0.0000,neutral
2,RT @BritishVogue: Have you seen #TheGirlOnTheT...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",35,rt seen yet here s need know watching film,0.0000,neutral
3,RT @girlontrainfilm: #TheGirlOnTheTrain is num...,en,None,Aron Adimin,None,"Liwutung, Sulawesi Utara",62,rt number film uk seen yet,0.0772,positive
4,#thegirlonthetrain #rainydayread #thankyouomeg...,en,None,Alle Davis,None,"New Plymouth District, New Zea",0,bekahs day b made,0.0000,neutral
5,Just saw #TheGirlOnTheTrain Emily Blunt's perf...,en,None,Heather K.,None,"North Carolina, USA",0,saw emily blunts performance amazing,0.5859,positive
6,RT @MissCharleyF: Wow! @PaulaHWrites' #TheGirl...,en,None,craig a ball,None,,1,rt wow outstanding thriller full twist turn su...,0.8481,positive
7,Finally saw #TheGirlOnTheTrain @TGOTTfilm @emi...,en,None,Carrie Tyrie,None,"Scotland, United Kingdom",0,finally saw amazing imagined,0.5859,positive
8,yay almost done w reading my book! took advant...,en,None,Saritah,None,AZ,0,yay almost done w reading book took advantage ...,0.6597,positive
9,"Loved, loved the book: #thegirlonthetrain Jus...",en,None,Sheila Tidwell,None,"Nashville, TN",0,loved loved book saw movie and,0.8316,positive


In [99]:
tweets.to_csv("/Users/uun466/Desktop/Data-Science-Project/tweet_file.csv", encoding = 'utf-8')

In [100]:
df = sqlContext.read.load('/Users/uun466/Desktop/Data-Science-Project/tweet_file.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [101]:
df.show()

+---+--------------------+----+--------------+----------------+-----------+--------------------+--------------+--------------------+---------------+---------+
|   |                text|lang|       country|         user_nm|coordinates|            location|retweets_count|          text_clean|sentiment_score|sentiment|
+---+--------------------+----+--------------+----------------+-----------+--------------------+--------------+--------------------+---------------+---------+
|  0|RT @finaldraftinc...|  en|              |   Ian Jon McCoy|           |    Atlanta, Georgia|             3|rt erin cressida ...|         0.1027| positive|
|  1|I don't understan...|  en|              | Bourlart Noemie|           |       Lille, France|             0|dont understand m...|            0.0|  neutral|
|  2|RT @BritishVogue:...|  en|              |     Aron Adimin|           |Liwutung, Sulawes...|            35|rt seen yet here ...|            0.0|  neutral|
|  3|RT @girlontrainfi...|  en|              |